# Gathering the data

### Gathering the data from the csv file by pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests as rq
import tweepy
import json


In [ ]:
from PIL import Image
from io import BytesIO

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
df  = pd.read_csv("twitter-archive-enhanced.csv")

In [ ]:
df.head()


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending = False)/ df.shape[0]

In [ ]:
df['floofer'].value_counts()

### Gathering the data from the tsv file by requests

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

r =  rq.get(url)

with open('image-predictions.tsv', 'wb') as file:
    file.write(r.content)

df1 = pd.read_csv("image-predictions.tsv", sep= '\t')

tweet_ids = df1['tweet_id'].tolist()
tweet_ids


### Gathering data from twitter by json

##### I tried to gather the data from twitter, but there was a problem in the access and Erorr 403 alwayes appear and I was not able to fix it. 

#### I downloaded the tweet_json.txt file from the classroom in Udacity

In [ ]:
tweet_data = []

# Read the JSON data from the file
with open('tweet_json.txt', 'r') as file:
    for line in file:
        tweet = json.loads(line)
        tweet_data.append(tweet)


df_tweets = pd.DataFrame(tweet_data)


In [ ]:
df_tweets.info()

# Access the data 

### Qualtiy issues

#### 1) In the first dataset, there are 5 columns that need to drop as they contain Nans more than 90%.
#### 2) timestamp is object and need to convert into datetime.
#### 3) tweet_id in the first and second datasets is int64 and need to convert into string.
#### 4) In the third dataset there are about 13 columns that need to drop as they contain Nans more than 90%.
#### 5) In the third dataset extended entities and user columns contain the same data (id and id str) in the dataframe so we can drop them.
#### 6) In the third dataset created at column needs to convert date into datetime.
#### 7) In the third dataset "id" and "id_str" columns are almost duplicated so we can drop id column as it is int64 and it is better for id to be a string.
#### 8) There is a row contain zero in the rating_denominator and after look to all the row, most of contents are Nans so it is better to drop the row.
#### 9) In the third dataset there are 6 columns that contain just one value which is False so we do not need it in the analysis.

In [ ]:
df.isnull().sum().sort_values(ascending = False)/ df.shape[0]

In [ ]:
df.info()

In [ ]:
df1.info()

In [ ]:
df1.head()

In [ ]:
df_tweets['truncated'].value_counts()

In [ ]:
df_tweets['retweeted'].value_counts()

In [ ]:
df_tweets['possibly_sensitive_appealable'].value_counts()

In [ ]:
df_tweets.head()

In [ ]:
df_tweets.isnull().sum().sort_values(ascending = False)/ df.shape[0]

In [ ]:
print(df_tweets['created_at'].dtypes)



### Tidiness issues

#### 1) In the first dataset the last four columns need to convert into one column "type of the dog" and put each type in the same row with the dog id 
#### 2) In the third dataset entieties column contain data as dictionry so it is better to extract these information into other column

# Clean the data

In [ ]:
df_copy = df.copy()
df1_copy = df1.copy()
df_tweets_copy = df_tweets.copy()

### Tidiness Issues

#### Define 1
#### 1) In the first dataset the last four columns need to convert into one column "type of the dog" and put each type in the same row with the dog id 

#### Code 1

In [ ]:
df_copy['dog_type'] = None

In [ ]:

def replace(typ):
    for index, i in df[typ].items():
        if i == typ:
            if pd.isna(df_copy.at[index, 'dog_type']):
                df_copy.at[index, 'dog_type'] = typ


replace('doggo')
replace('floofer')
replace('pupper')
replace('puppo')

##### Test 1

In [ ]:
df_copy['dog_type'].value_counts()

In [ ]:
# Now i can drop these columns
df_copy.drop(['doggo', 'floofer','pupper','puppo'], axis=1, inplace= True)


In [ ]:
df_copy.info()

##### Define 2

#### 2) In the third dataset entieties column contain data as dictionry so it is better to extract these information into other column

##### Code 2

In [ ]:
def extract_hashtags(entities):
    hashtags = entities.get('hashtags', [])
    return [hashtag['text'] for hashtag in hashtags]

# Apply the function to the 'entities' column
df_tweets_copy['hashtags'] = df_tweets_copy['entities'].apply(extract_hashtags)


print(df_tweets_copy[['hashtags']])


# the cell will not work more as i delete entities column

##### Test 2

In [ ]:
# Now i can drop entities column

df_tweets_copy.drop('entities', axis=1, inplace= True)

In [ ]:
df_tweets_copy.head(6)

### Quality issues

##### Define 1
##### 1) In the first dataset, there are 5 columns that need to drop as they contain Nans more than 90%.

##### Code 1

In [ ]:
# to see which columns need to drop

df_copy.isnull().sum().sort_values(ascending = False)/ df_copy.shape[0]

In [ ]:
# Filter out rows where retweet-related columns are not null
df_copy = df_copy[df_copy['retweeted_status_id'].isnull() & 
                     df_copy['retweeted_status_user_id'].isnull() & 
                     df_copy['retweeted_status_timestamp'].isnull()]


In [ ]:
df_copy.drop(['in_reply_to_user_id', 'in_reply_to_status_id','retweeted_status_timestamp','retweeted_status_id','retweeted_status_user_id'], axis=1, inplace= True)

##### Test 1

In [ ]:
# df_copy without the 5 columns 


df_copy.isnull().sum().sort_values(ascending = False)/ df_copy.shape[0]

##### Define 2
#### 2) timestamp is object and need to convert into datetime.

##### Code 2

In [ ]:
df_copy['timestamp'] = pd.to_datetime(df_copy['timestamp'])

##### Test 2

In [ ]:
print(df_copy['timestamp'].dtypes)

##### Define 3
#### 3) tweet_id in the first and second datasets is int64 and need to convert into string.

##### Code 3

In [ ]:
df_copy['tweet_id'] = df_copy['tweet_id'].astype(str)
df1_copy['tweet_id'] = df1_copy['tweet_id'].astype(str)


##### Test 3

In [ ]:
print(df_copy['tweet_id'].dtypes)
print(df1_copy['tweet_id'].dtypes)

##### Define 4
#### 4) In the third dataset there are about 13 columns that need to drop as they contain Nans more than 90%.

##### Code 4

In [ ]:
df_tweets_copy.drop(['geo','coordinates','contributors','place','quoted_status','quoted_status_id_str','quoted_status_id','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str','in_reply_to_screen_name','retweeted_status'], axis=1, inplace= True)

##### Test 4

In [ ]:
df_tweets_copy.isnull().sum().sort_values(ascending=False) / df_tweets_copy.shape[0]


##### Define 5
#### 5) In the third dataset extended entities and user columns contain the same data (id and id str) in the dataframe so we can drop them.


##### Code 5

In [ ]:
df_tweets_copy.drop(['extended_entities', 'user'], axis=1, inplace= True)

##### Test 5

In [ ]:
df_tweets_copy.head(6)

##### Define 6
#### 6) In the third dataset created at column needs to convert date into datetime.


##### Code 6

In [ ]:
df_tweets_copy['created_at'] = pd.to_datetime(df_tweets_copy['created_at'], format='%a %b %d %H:%M:%S %z %Y')

##### Test 6

In [ ]:
df_tweets_copy.info()

##### Define 7
#### 7) In the third dataset "id" and "id_str" columns are almost duplicated so we can drop id column as it is int64 and it is better for id to be a string.


##### Code 7

In [ ]:
df_tweets_copy.drop(['id'], axis = 1, inplace= True)

##### Test 7

In [ ]:
df_tweets_copy.head()

##### Define 8
##### 8) There is a row contain zero in the rating_denominator and after look to all the row, most of contents are Nans so it is better to drop the row.

##### Code 8

In [ ]:
df_copy.head()

In [ ]:
df_copy[df_copy['rating_denominator']== 0]

In [ ]:
df_copy.drop(index= 313 ,axis = 0 , inplace= True)

##### Test 8

In [ ]:
df_copy[df_copy['rating_denominator']== 0]

##### Define 9
#### 9) In the third dataset there are 6 columns that contain just one value which is False so we do not need it in the analysis.

##### Code 9

In [ ]:
df_tweets_copy.drop(['truncated', 'retweeted','possibly_sensitive','possibly_sensitive_appealable','favorited','is_quote_status'], axis=1, inplace= True)

##### Test 9

In [ ]:
df_tweets_copy.head()

### Now look to the three pieces of data after cleaning

In [ ]:
df_copy.head()

In [ ]:
df1_copy.head()

In [ ]:
df_tweets_copy.head()

In [ ]:
# rename the id_str with tweet_id

df_tweets_copy.rename(columns={'id_str': 'tweet_id'}, inplace= True)

### Final dataframe

In [ ]:
df_combined = pd.merge(df_copy, df_tweets_copy, on='tweet_id')

# Merge with df1_copy
df_final = pd.merge(df_combined, df1_copy, on='tweet_id', how='left')

In [ ]:
df_final.head()

# Store the data 

In [ ]:
df_final.to_csv('twitter_archive_master.csv', index=False)

# Analyzing and Visualizing Data

In [ ]:
df_final.head(2)

### Insights and their visualization

#### 1) Most Common Dog Breeds in Tweets

#### Data Used: p1, p2, p3 columns which indicate the predicted dog breeds from the image.

In [ ]:
all_breeds = pd.concat([df_final['p1'], df_final['p2'], df_final['p3']])
common_breeds = all_breeds.value_counts().head(20)

# Visualization
plt.figure(figsize=(10, 5))
common_breeds.plot(kind='bar')
plt.title('Most Common Dog Breeds in Tweets')
plt.xlabel('Dog Breed')
plt.ylabel('Frequency')
plt.show()


#### From the graph, the most commom dog breeds are golden_retriever and Labrador_retriever

#### 2) Relationship Between Retweet Count and Favorite Count

#### Data Used: retweet_count and favorite_count columns.

In [ ]:
df_final['retweet_count'].isnull().sum()
df_final['favorite_count'].isnull().sum()

In [ ]:
df_final['retweet_count'] = pd.to_numeric(df_final['retweet_count'], errors='coerce')
df_final['favorite_count'] = pd.to_numeric(df_final['favorite_count'], errors='coerce')


In [ ]:
# There is very strong correation between retweet and favourite count with 0.91

correlation = df_final['retweet_count'].corr(df_final['favorite_count'])

# Visualization
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_final, x='retweet_count', y='favorite_count', color='blue')
plt.title('Relationship Between Retweet Count and Favorite Count')
plt.xlabel('Retweet Count')
plt.ylabel('Favorite Count')
plt.grid(True)
plt.tight_layout()
plt.show()

#### 3) Most Popular Tweet Sources

#### Data Used: source_y column.

In [ ]:
# Cleaning the source column to get meaningful names
df_final['source'] = df_final['source_y'].str.extract(r'>([^<]+)<')

# Counting the frequency of each source
source_counts = df_final['source'].value_counts()

# Plotting the bar chart
plt.figure(figsize=(12, 6))
source_counts.plot(kind='bar', color='skyblue')
plt.title('Most Popular Tweet Sources')
plt.xlabel('Source')
plt.ylabel('Number of Tweets')
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_final.head()